In [ ]:
# -*- coding: utf-8 -*-
# install libraries
%pip install --upgrade tiktoken
%pip install --upgrade openai
%pip install --upgrade python-dotenv
%pip install --upgrade pydub
%pip install --upgrade soundfile
%pip install --upgrade openai-whisper

In [2]:
# import libraries
import os
import re
import time
import pprint
import tiktoken
import concurrent.futures
from dotenv import load_dotenv
from openai import AzureOpenAI
import openai
import pandas as pd

# Funções que serão utilizadas durante os exemplos

In [3]:
# Nome das regiões Azure OpenAI
EASTUS = "eastus"
CANADAEAST = "canadaeast"
NORTHCENTRALUS = "northcentralus"

# Nome dos modelos Azure OpenAI criados na subscrição 
GPT_35_TURBO = "gpt-35-turbo"
GPT_35_TURBO_16K = "gpt-35-turbo-16k"
GPT_35_TURBO_INSTRUCT = "gpt-35-turbo-instruct"

GPT_4 = "gpt-4"
GPT_4_32K = "gpt-4-32k"
GPT_4_TURBO = "gpt-4-turbo"
GPT_4o = "gpt-4o"
GPT_4o_mini = "gpt-4o-mini"

DALL_E_3 = "dall-e-3"
WHISPER = "whisper"
TEXT_EMBEDDING_ADA_002 = "text-embedding-ada-002"
TEXT_EMBEDDING_ADA_3_SMALL = "text-embedding-3-small"
TEXT_EMBEDDING_ADA_3_LARGE = "text-embedding-3-large"

# Whisper limit 25MB files
audio_chunk_size_kb = 1024 * 22


load_dotenv() # carregar variáveis de ambiente

deployments_in_regions = {"CANADAEAST": [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO, TEXT_EMBEDDING_ADA_002, TEXT_EMBEDDING_ADA_3_SMALL],
                          "EASTUS": [GPT_35_TURBO_16K, GPT_35_TURBO_INSTRUCT, GPT_4o, GPT_4o_mini, DALL_E_3, TEXT_EMBEDDING_ADA_002, TEXT_EMBEDDING_ADA_3_LARGE],
                          "NORTHCENTRALUS": [WHISPER, GPT_4o]}


# carregar tokenizador para os modelos de linguagem
encoding = tiktoken.get_encoding("cl100k_base")

# Exemplos de system messages
papel = {"bot": "Aja como um assistente virtual, capaz de responder perguntas, fornecer informações e que sempre responde em formato JSON.",
        "escritor": "Aja como um especialista em escrita e redação, capaz de escrever, compreender e analisar textos em português, especialmente textos opinativos e argumentativos.",
        "repentista": "Aja como um repentista, capaz de improvisar versos e rimas em português, especialmente em forma de sextilhas e martelos agalopados.",
        "legislador": "Aja como um especialista em legislação, capaz de escrever, interpretar e analisar leis e normas jurídicas em português.",
        "comediante": "Aja como um comediante, capaz de criar piadas, contar histórias engraçadas e fazer humor em português.",
        "desevolvedor": "Aja como um desenvolvedor de software, capaz de criar, testar e manter programas de computador em qualquer linguagem, mas em especial em Python, HTML e Javascript",
        "hacker": "Aja como um hacker, capaz de invadir sistemas de computadores, roubar informações e burlar sistemas de segurança.",
        "professor": "Aja como um professor, capaz de ensinar e explicar conceitos, teorias e práticas em qualquer disciplina. Com explicações claras e didáticas, com exemplos práticos e exercícios para fixação dos conceitos.",
        "instrutor": "Aja como um instrutor, capaz de ensinar e treinar pessoas em qualquer área de conhecimento. Suas explicações são diretas, sucintas e sem rodeios.",
        "medico": "Aja como um médico, capaz de diagnosticar, tratar e prevenir doenças em seres humanos e animais.",
        "advogado": "Aja como um advogado, capaz de representar clientes em processos judiciais, elaborar contratos e prestar consultoria jurídica.",
        "cartomante": "Aja como um cartomante, capaz de ler cartas de tarô, búzios e outros oráculos, interpretar sonhos e prever o futuro e o destino das pessoas.",
        "astrólogo": "Aja como um astrólogo, capaz de interpretar mapas astrais, prever o futuro e o destino das pessoas com base na posição dos astros e dos signos do zodíaco.",
        "marketeiro": "Aja como um especialista em marketing, profundo conhecedor de branding, construção de marcas, propaganda e publicidade.",
        "padre": "Aja como um padre, capaz de ministrar sacramentos, celebrar missas e aconselhar fiéis em questões espirituais.",
        "rabino": "Aja como um rabino, capaz de ministrar cerimônias religiosas, ensinar a Torá e aconselhar fiéis em questões espirituais.",
        "pastor": "Aja como um pastor, capaz de ministrar cultos, pregar a Bíblia e aconselhar fiéis em questões espirituais.",}



# inicialização do cliente Azure OpenAI Canada East
client_canadaeast = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_CANADAEAST"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_CANADAEAST"),  
    api_version="2024-02-15-preview"
)

# inicialização do cliente Azure OpenAI East US
client_eastus = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_EASTUS"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_EASTUS"),  
    api_version="2024-02-15-preview"
)

# inicialização do cliente Azure OpenAI North Central US
client_northcentralus = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_NORTHCENTRALUS"), 
    api_key=os.getenv("AZURE_OPENAI_KEY_NORTHCENTRALUS"),  
    api_version=os.getenv("WHISPER_VERSION")
)


# funcao para retornar o cliente de acordo com a região
def get_client(region):
    if region == "canadaeast":
        return client_canadaeast
    elif region == "eastus":
        return client_eastus
    elif region == "northcentralus":
        return client_northcentralus
    else:
        return None

# função para chamar o modelo de linguagem
def call_llm(deployment_name, system_message, question, region="canadaeast"):
    #verificar se o modelo está disponível na região
    if deployment_name not in deployments_in_regions[region.upper()]:
        return {"0.error": f"Model {deployment_name} not available in region {region}", 
                "1.This regions has the following models available": deployments_in_regions[region.upper()], 
                f"2.The model {deployment_name} is available in the following regions": [region for region, models in deployments_in_regions.items() if deployment_name in models]}
    
    client = get_client(region)
    start_time = time.perf_counter()
    response = client.chat.completions.create(
        model = deployment_name,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ]
    )
    elapsed_time = time.perf_counter() - start_time # usado para calcular o tempo de execução
    resp = response.choices[0].message.content
    tokens_completion = num_tokens_from_string(resp, deployment_name) # conta os tokens da resposta
    # conta os tokens da mensagem do sistema e da pergunta (prompt)
    tokens_prompt = num_tokens_from_string(system_message, deployment_name) + num_tokens_from_string(question, deployment_name)
    # formatar a resposta da função
    return {"0.model": deployment_name, 
            "1.elapsed_time": elapsed_time, 
            "2.response": resp, 
            "3.num_tokens_completion": tokens_completion, 
            "4.num_tokens_prompt": tokens_prompt,
            "5.system_message": system_message,
            "6.question": question}

# funcao para contar tokens
def num_tokens_from_string(texto, model):
    if not texto:
        return 0
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(texto))
    return num_tokens

# função para gerar embeddings
def generate_embeddings(text, model, region="canadaeast"):
    client = get_client(region)
    start_time = time.perf_counter()
    embeddings = client.embeddings.create(input = [text], model=model).data[0].embedding
    elapsed_time = time.perf_counter() - start_time
    return embeddings, elapsed_time

# função para executar modelos simultaneamente
# models: lista de modelos
# system_message: mensagem do sistema
# question: pergunta
# retorna uma lista com os resultados
def execute_simultaneously(models, system_message, question, debug=False, region="canadaeast"):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(call_llm, model, system_message, question, region) for model in models]
        results = []
        for future in concurrent.futures.as_completed(futures):
            if debug:
                print_result(future.result(), debug)
            results.append(future.result())
    return results

def print_result(result, debug=False):
    print("Elapsed time:", result["1.elapsed_time"])
    print("Response:", result["2.response"])
    print("Num tokens completion:", result["3.num_tokens_completion"])
    print("Num tokens prompt:", result["4.num_tokens_prompt"])
    if debug:
        print("System message:", result["5.system_message"])
        print("Question:", result["6.question"])
        print("Model:", result["0.model"])
        print("===========================================================\n")  

#funcao para imprimir resultados
def print_results(system_message, question, results, debug=False):
    if debug:
        print("System message:", system_message)
        print("Question:", question)

    for result in results:
        print("Model:", result["0.model"], "Elapsed time:", result["1.elapsed_time"])
        print("Response:", result["2.response"])
        print("Num tokens completion:", result["3.num_tokens_completion"], 
              ", Num tokens prompt:", result["4.num_tokens_prompt"])
        print("===========================================================\n")


def load_log_file(file_path):
    # Definir um regex para capturar os campos do log
    log_pattern = r'(\S+) - - \[(.*?)\] "(.*?)" (\d{3}) (\d+) "(.*?)" "(.*?)" (\d+)'

    # Listas para armazenar as informações extraídas
    log_data = {
        'ip': [],
        'timestamp': [],
        'request': [],
        'status_code': [],
        'size': [],
        'referrer': [],
        'user_agent': [],
        'response_time': []
    }

    # Ler o arquivo linha por linha
    with open(file_path, 'r') as file:
        for line in file:
            match = re.match(log_pattern, line)
            if match:
                log_data['ip'].append(match.group(1))
                log_data['timestamp'].append(match.group(2))
                log_data['request'].append(match.group(3))
                log_data['status_code'].append(int(match.group(4)))
                log_data['size'].append(int(match.group(5)))
                log_data['referrer'].append(match.group(6))
                log_data['user_agent'].append(match.group(7))
                log_data['response_time'].append(int(match.group(8)))

    # Converter para DataFrame pandas
    df = pd.DataFrame(log_data)
    # Converter a coluna 'timestamp' para o tipo datetime, permitindo formatos mistos
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')
    
    return df

# Exemplo de uso
file_path = 'logfiles.log'  # Caminho para o arquivo de log
df_logs = load_log_file(file_path)


In [ ]:
def sample_agenda():
    agenda = '''12/03/2022 12:00:00 Ocupado 12/03/2022 13:00:00 Ocupado 12/03/2022 14:00:00 Ocupado 12/03/2022 15:00:00 Livre 12/03/2022 16:00:00 Ocupado 12/03/2022 17:00:00 Livre
        12/03/2022 18:00:00 Ocupado 12/03/2022 19:00:00 Livre 12/03/2022 20:00:00 Ocupado'''
    return call_llm(GPT_35_TURBO, papel["bot"], 
                    f"Com base nessa agenda, qual o próximo horário livre? {agenda}", "canadaeast")

resposta = sample_agenda()
print_result(resposta)

Elapsed time: 2.333334900002228
Response: {
  "próximo_horário_livre": "12/03/2022 15:00:00"
}
Num tokens completion: 26
Num tokens prompt: 183


# Exemplo de diferentes tons na resposta

In [5]:
def sample_system_message_intonation(question):
    system_message = {"despojado":"Você é um assitente que sempre responde em tom despojado, de forma didática e em porguguês, apesar de eventualmente citar palavras em inglês.",
                      "formal":"Você é um assistente que sempre responde em tom formal, rebuscado, sem citar palavras em inglês.",
                      "informal":"Você é um assistente que sempre responde em tom informal, de forma didática e em português, com gírias e expressões coloquiais."}
    

    for item in system_message:
        print(f"Resposta com tom {item.upper()}:")
        r = call_llm(GPT_35_TURBO, system_message[item], question, "canadaeast")
        pprint.pprint(r["2.response"])

sample_system_message_intonation("O que é inteligência artificial?")

Resposta com tom DESPOJADO:
('Inteligência artificial é tipo quando máquinas conseguem aprender e realizar '
 'tarefas que normalmente precisariam do pensamento humano. É como ensinar um '
 'computador a reconhecer imagens, entender linguagem, ou até mesmo tomar '
 'decisões por conta própria. IA pode ser usada em muitos lugares, tipo em '
 'carros autônomos, assistentes virtuais, ou até na recomendação de produtos '
 'em sites de compras. Resumindo, é quando as máquinas mostram um jeitinho de '
 'pensar!')
Resposta com tom FORMAL:
('A inteligência artificial é o ramo da ciência da computação que se dedica ao '
 'desenvolvimento de sistemas e técnicas capazes de simular o processo de '
 'pensamento humano, realizando tarefas que exigem decisões baseadas em '
 'raciocínio, aprendizado e percepção. Esses sistemas são projetados para '
 'realizar funções que, de outra forma, exigiriam a intervenção humana, a '
 'partir de algoritmos e modelos matemáticos que permitem a tomada de decisões 

# Exemplo de diferença entre modelos

In [ ]:
system_message = papel["instrutor"]
question = "Conte-me um paradoxo de maneira estrudurada e sucinta com o mínimo de detalhes possível."
models = [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO]
results = execute_simultaneously(models, system_message, question, True)
print_results(system_message, question, results)

In [ ]:
# faça uma pergunta ao modelo, seja criativo!
system_message = papel["astrólogo"]

prompt = ''' 

Diga o horóscopo do signo do dia de hoje para o signo de Áries.

'''

print(call_llm(GPT_4o, system_message, prompt, EASTUS)["2.response"])


In [25]:
file_path = 'logfiles.log'  # Caminho para o arquivo de log
df_logs = load_log_file(file_path)
df_logs

,ip,timestamp,request,status_code,size,referrer,user_agent,response_time
0,107.160.167.84,2023-02-17 19:05:01.237,DELETE /usr HTTP/1.0,200,5028,-,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,965
1,51.158.33.132,2023-02-17 19:05:01.802,GET /usr/login HTTP/1.0,500,4954,-,Mozilla/5.0 (Android 10; Mobile; rv:84.0) Geck...,2929
2,124.126.180.201,2023-02-17 19:05:01.820,POST /usr/admin HTTP/1.0,403,4970,http://davis.com/blog/wp-content/postslogin.html,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1962
3,43.168.19.42,2023-02-17 19:05:02.500,DELETE /usr/register HTTP/1.0,403,5055,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:8...,702
4,88.224.29.254,2023-02-17 19:05:02.768,POST /usr/admin/developer HTTP/1.0,304,4990,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,4130
...,...,...,...,...,...,...,...,...
9995,152.193.37.56,2023-02-17 20:28:36.959,DELETE /usr/admin HTTP/1.0,404,5008,http://davis.com/blog/wp-content/postslogin.html,Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_9 like...,3775
9996,200.209.229.252,2023-02-17 20:28:37.569,DELETE /usr HTTP/1.0,502,4951,-,Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_9 like...,590
9997,223.58.153.52,2023-02-17 20:28:37.774,PUT /usr/admin HTTP/1.0,200,4968,-,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,1633
9998,201.232.185.137,2023-02-17 20:28:38.766,PUT /usr/admin/developer HTTP/1.0,502,5062,-,Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000)...,3009


In [28]:
# 200- Ok, 303 - See Other, 304 - Not Modified, 403 - Forbidden , 404 - Not Found, 500 - Internal Server Error, 502 - Bad Gateway, 

df_selected = df_logs[['status_code', 'response_time']]
df_as_csv = df_selected.to_csv()

prompt = f'''
com base no dataset fornecido a seguir, quais são os insights que você pode fornecer?

Dataset:

{df_as_csv}

'''
system_message = "Aja como um assistente virtual, capaz de responder perguntas, fornecer informações sobre datasets de logs e ofercer insights sobre os dados."

r = call_llm(GPT_4_TURBO, system_message, prompt, CANADAEAST)

In [29]:
print(r["2.response"])

Este dataset contém uma lista de registros de resposta a solicitações HTTP com os campos de código de status (status_code), e tempo de resposta (response_time) em milissegundos. Com base nisso, aqui estão alguns insights que podem ser observados nos dados:

1. **Distribuição dos Códigos de Estado (status_code)**:
   - Podemos verificar a frequência de cada tipo de código de status para entender como o servidor está respondendo. Por exemplo, um alto número de códigos de status na faixa de 500 pode indicar problemas do lado do servidor, enquanto muitos códigos 404 podem indicar muitos links quebrados ou erros no cliente.

2. **Tempo Médio de Resposta**:
   - Calcular o tempo médio de resposta pode ajudar a entender a eficiência do servidor. Tempos de resposta elevados podem ser um indicativo de problemas de desempenho.

3. **Análise de Outliers de Tempo de Resposta**:
   - Identificar outliers, ou seja, tempos de resposta excepcionalmente altos ou baixos, pode ajudar a identificar possív